In [ ]:
import basf2 as b2
import modularAnalysis as ma
import variables.collections as vc
import variables.utils as vu
import stdCharged as stdc
import variables as va
from stdV0s import stdKshorts
from stdPi0s import stdPi0s
from stdPhotons import stdPhotons
from datetime import *
from ROOT import Belle2
import vertex as vx
import os
import math
import sys

In [ ]:
my_path = b2.create_path()
b2.set_log_level(b2.LogLevel.ERROR)

In [ ]:
input_file = []
output_file = "out_CFT.root"
ma.inputMdstList(input_file, path=my_path)

In [ ]:
ma.fillParticleList(
    "pi+:D0", "thetaInCDCAcceptance and dr < 1 and abs(dz) < 3", path=my_path
)
ma.fillParticleList(
    "K+:D0", "thetaInCDCAcceptance and dr < 1 and abs(dz) < 3", path=my_path
)

In [ ]:
ma.reconstructDecay(
    decayString="D0:full -> K-:D0 pi+:D0",
    cut="1.82 < InvM < 1.90",
    path=my_path,
)

In [ ]:
ma.matchMCTruth(list_name="D0:Kpi", path=my_path)

In [ ]:
vx.treeFit('D0:Kpi',0.001,path=my_path)

In [ ]:
va.variables.addAlias("genGrandmotherPDG", "genMotherPDG(1)")
va.variables.addAlias("binaryPID_K_pi", "binaryPID(321,211)")
va.variables.addAlias("binaryPID_mu_pi", "binaryPID(13,211)")

In [ ]:
kinematics = ["pt", "p", "E","cosTheta","theta","phi"]
cms_kinematics = vu.create_aliases(kinematics, "useCMSFrame({variable})", "CMS")

In [ ]:
other_sig = ["M","genMotherPDG","genGrandmotherPDG","charge","protonID","electronID","kaonID","pionID","muonID"]

In [ ]:
ma.buildRestOfEvent('D0:Kpi', path=my_path)

In [ ]:
cleanMask = ('cleanMask', 'dr < 1 and abs(dz) < 3','')
ma.appendROEMasks('D0:Kpi', [cleanMask], path = my_path)

In [ ]:
d_vars = kinematics+cms_kinematics+vc.mc_truth+["M","genMotherPDG","charge"]
vars = vu.create_aliases_for_selected(
    list_of_variables=vc.mc_truth
    + kinematics
    + cms_kinematics
    + other_sig,
    decay_string="D0 -> ^K- ^pi+",
)

In [ ]:
ma.variablesToNtuple(
    "D0:Kpi",
    d_vars+vars,
    filename=output_file,
    treename="D0tree",
    path=my_path,
)

In [ ]:
b2.process(my_path)